<a href="https://colab.research.google.com/github/cmbhatt1/singaporeResaleFlat/blob/main/ResaleFlatPricesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Singapore Resale Flat Prices Prediction**

**Problem Statement:**
The objective of this project is to develop a machine learning model and deploy it as a user-friendly web application that predicts the resale prices of flats in Singapore. This predictive model will be based on historical data of resale flat transactions, and it aims to assist both potential buyers and sellers in estimating the resale value of a flat.


*We start with processing the data. Then, we will build a model. This is a regression task and multiple algorithms(Decision Trees, Random Forests, Linear Regression and XGboost) have been used to calculate the most accurate model. This model will be used to predict prices based on users requirements in a streamlit app*

We have 5 csv sheets which contains the resale data from 1990 to 2024.

In [123]:
!pip install streamlit
!pip install scikit-learn
!pip install geopy

The files are available [here](https://beta.data.gov.sg/collections/189/datasets/d_ebc5ab87086db484f88045b47411ebc5/view)

In [124]:
import pandas as pd
resale_1990_1999 = pd.read_csv('ResaleFlatPricesBasedonApprovalDate19901999.csv')
resale_2000_2012 = pd.read_csv('ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv')
resale_2015_2016 = pd.read_csv('ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
resale_2012_2014 = pd.read_csv('ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv')
resale_2017_2024 = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

resale_data = pd.concat([resale_1990_1999, resale_2000_2012, resale_2012_2014, resale_2015_2016, resale_2017_2024])
resale_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [125]:
# Convert 'remaining_lease' column to string type where it's not null
resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'] = resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'].astype(str)

# Split the string and take the first element (as some of the data is in the format "63 years and 4 months")
resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'] = resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'].str.split().str[0]

# Convert the remaining_lease column to integer type
resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'] = resale_data.loc[~resale_data['remaining_lease'].isnull(),'remaining_lease'].astype(int)

# Convert the month column to datetime type
resale_data.rename(columns={'month': 'year_sold'}, inplace=True)

resale_data["year_sold"] = pd.to_datetime(resale_data["year_sold"], format = '%Y-%m')

# Extract year and month from the month column
resale_data['year'] = resale_data['year_sold'].dt.year

resale_data["month_1"] = resale_data['year_sold'].dt.month

resale_data["lease_commence_date"] = pd.to_datetime(resale_data["lease_commence_date"])

# Since all the values where remaining_lease was present added up to 99(month - ), we fill empty ones out as well.
resale_data.loc[resale_data['remaining_lease'].isnull(),'remaining_lease'] = 99-resale_data.loc[resale_data['remaining_lease'].isnull(),'year']+resale_data.loc[resale_data['remaining_lease'].isnull(),'lease_commence_date'].dt.year

## Fetching Latitutde and Longitude
Most ML algos work better with numerical data. Hence, we will use latitude and longitude to replace the street, town and block columns.

First, we will fetch the latitude and longitudes of the individual entries via geopy library.
This library gives the latitude and longitude of the place and requires the data to be in the form street, town, country.
We will combine street, town and add "Singapore" to it and save it in addreses array(add_arr). After this, let's get all the unique addresses.

This list is fed to the get_coordinates function to get the latitude and longitudes of the place.

This will take a few minutes to run and hence, I have uploaded it to my git repo(coordinates_data.csv). If you would like to run this cell anyway, please remove the comments and comment this line

> coordinates_df = pd.read_csv('coordinates_data.csv')





In [126]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="test1")

# def get_coordinates(street_address):
#     geolocator = Nominatim(user_agent="test1", timeout=10)
#     location = geolocator.geocode(street_address)
#     if location:
#         return (location.latitude, location.longitude)
#     else:
#         return None

# coordinates_list = []

# add_arr = resale_data['street_name'].astype(str).values +','+ resale_data['town'].astype(str).values + ', Singapore'

# unique_add_arr = list(set(add_arr))

# for street in unique_add_arr:
#   coordinates = get_coordinates(street)
#   if coordinates:
#     row = {'Address': street,'Latitude': coordinates[0], 'Longitude': coordinates[1]}
#     coordinates_list.append(row)
#   else:
#     update_street = street.split(',',1)
#     coordinates = get_coordinates(update_street[1])
#     if coordinates:
#       row = {'Address': street,'Latitude': coordinates[0], 'Longitude': coordinates[1]}
#       coordinates_list.append(row)
#     else:
#       row = {'Address': street,'Latitude': 'NA', 'Longitude': 'NA'}
#       coordinates_list.append(row)

# coordinates_df=pd.DataFrame(coordinates_list)
## After analyzing the data, we see that most missing values correspond to a single town.
## Since, we don't have the street names for these entries we impute the
# coordinates_df.loc[coordinates_df['Latitude']=='NA',["Latitude","Longitude"]] = 1.28967, 103.85007
coordinates_df = pd.read_csv('coordinates_data.csv')
coordinates_df.head()

,full_address,Latitude,Longitude
0,"SUMANG WALK,PUNGGOL, Singapore",1.404233,103.894901
1,"BT BATOK WEST AVE 4,BUKIT BATOK, Singapore",1.349057,103.749591
2,"WOODLANDS DR 71,WOODLANDS, Singapore",1.439742,103.800216
3,"BISHAN ST 24,BISHAN, Singapore",1.357512,103.845095
4,"AH HOOD RD,KALLANG/WHAMPOA, Singapore",1.331646,103.838905


In [127]:
print(resale_data['flat_type'].value_counts())
print(resale_data['flat_model'].value_counts())

flat_type
4 ROOM              350982
3 ROOM              295146
5 ROOM              194721
EXECUTIVE            69449
2 ROOM               11753
1 ROOM                1303
MULTI GENERATION       279
MULTI-GENERATION       264
Name: count, dtype: int64
flat_model
Model A                   193470
Improved                  167476
New Generation            109708
NEW GENERATION             78898
IMPROVED                   73589
MODEL A                    70381
Premium Apartment          46470
Simplified                 34182
Apartment                  25468
Standard                   25085
SIMPLIFIED                 23258
STANDARD                   17375
Maisonette                 17364
MAISONETTE                 12215
Model A2                   10108
APARTMENT                   9901
DBSS                        3269
Adjoined flat               1244
Model A-Maisonette          1089
MODEL A-MAISONETTE           982
Terrace                      445
Type S1                      434
MULTI GENER

## Merging the data and doing some preprocessing steps

1) We will merge the data on full_address column.

2) Converting the data type of month column to DateTime

3) Flat_type col contains the same type('MULTI GENERATION') which has different formatting. So, that too is changed.

4) Conversion of flat_model cols to lower case.

5) Since we are utilizing lat and long, we do not need the block, street_name, full_address and town column.
Most of the entries inside reamining lease and lease_commence_data are empty, so those 2 are also dropped.



In [128]:
resale_data['full_address'] = resale_data['street_name'].astype(str).values +','+ resale_data['town'].astype(str).values + ', Singapore'

resale_data_latlong = pd.merge(resale_data, coordinates_df, on='full_address', how='left')

resale_data_latlong['flat_type'] = resale_data_latlong['flat_type'].replace('MULTI GENERATION','MULTI-GENERATION')
resale_data_latlong['flat_model'] = resale_data_latlong['flat_model'].str.lower()

resale_data_latlong = resale_data_latlong.drop(["street_name","block","full_address","town","year","month_1"],axis=1)
resale_data_latlong.head()

,year_sold,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,Latitude,Longitude
0,1990-01-01,1 ROOM,10 TO 12,31.0,improved,1970-01-01 00:00:00.000001977,9000.0,79,1.360174,103.852629
1,1990-01-01,1 ROOM,04 TO 06,31.0,improved,1970-01-01 00:00:00.000001977,6000.0,79,1.360174,103.852629
2,1990-01-01,1 ROOM,10 TO 12,31.0,improved,1970-01-01 00:00:00.000001977,8000.0,79,1.360174,103.852629
3,1990-01-01,1 ROOM,07 TO 09,31.0,improved,1970-01-01 00:00:00.000001977,6000.0,79,1.360174,103.852629
4,1990-01-01,3 ROOM,04 TO 06,73.0,new generation,1970-01-01 00:00:00.000001976,47200.0,79,1.360174,103.852629


## Splitting Data and encoding the categorical columns

1) Stratified shuffle split to the resale_data_latlong table on the year column as both the test and train data must contain equal proportions of years.

2) flat_type and storey_range are ordinal categorical columns and are transformed with the help of ordinal encoder.

3) flat_model on the other hand is nominal, so it is transformed using OneHotencoding.

4) We will transform the features into an array and concatenate with flat_model_1hot array.



In [129]:
resale_data_latlong['year_sold'] = resale_data_latlong['year_sold'].apply(lambda x: x.toordinal())
resale_data_latlong['lease_commence_date'] = resale_data_latlong['lease_commence_date'].apply(lambda x: x.toordinal())

In [130]:
resale_data_latlong.head()

,year_sold,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,Latitude,Longitude
0,726468,1 ROOM,10 TO 12,31.0,improved,719163,9000.0,79,1.360174,103.852629
1,726468,1 ROOM,04 TO 06,31.0,improved,719163,6000.0,79,1.360174,103.852629
2,726468,1 ROOM,10 TO 12,31.0,improved,719163,8000.0,79,1.360174,103.852629
3,726468,1 ROOM,07 TO 09,31.0,improved,719163,6000.0,79,1.360174,103.852629
4,726468,3 ROOM,04 TO 06,73.0,new generation,719163,47200.0,79,1.360174,103.852629


In [131]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1,test_size = 0.2,random_state=42)
strat_splits = []
for train_index, test_index in splitter.split(resale_data_latlong, resale_data_latlong['year_sold']):
  train_data = resale_data_latlong.iloc[train_index]
  test_data = resale_data_latlong.iloc[test_index]
  strat_splits.append([train_data, test_data])


from sklearn.preprocessing import OrdinalEncoder
flat_type_order = [['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM','EXECUTIVE','MULTI-GENERATION']]
storey_range_order = [['01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09', '10 TO 12', '11 TO 15','13 TO 15', '16 TO 18', '16 TO 20', '19 TO 21',
                       '21 TO 25', '22 TO 24', '25 TO 27','26 TO 30', '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40', '37 TO 39','40 TO 42',
                       '43 TO 45', '46 TO 48', '49 TO 51']]
ord_encoder_flattype = OrdinalEncoder(categories=flat_type_order)
ord_encoder_storey = OrdinalEncoder(categories=storey_range_order)
train_data.loc[:,"flat_type"] = ord_encoder_flattype.fit_transform(train_data[["flat_type"]])
train_data.loc[:,"storey_range"] = ord_encoder_storey.fit_transform(train_data[["storey_range"]])

from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder(handle_unknown="ignore")
flat_model_1hot = cat_encoder.fit_transform(train_data[["flat_model"]])

train_data = train_data.drop("flat_model",axis = 1)

import numpy as np
target = train_data['resale_price'].values
train_data = train_data.drop("resale_price",axis=1)
features = np.concatenate((train_data.values,flat_model_1hot.toarray()),axis=1)


## Regression models

We will be using RMSE as a metric to compute the accuracy of model. The train dataset will be fit on multiple models(Decision tree regressor, Random forest regressor 1hot, Random forest freq, Linear regression, bagging regressor) and for each model the train and test RMSE will be calculated. This will be saved in a dataframe named performance of models. We will then decide which is the best model to fit the data on.

The difference between Random forest 1hot and random forest freq is that the former uses the flat_model column transformed to 1hot array whereas the latter uses it after transformation to frequency encoded column. This is because Random forest doesn't work well with 1hot encoded data.

The next code cell can be commented out if you do not want to run all these models.

In [132]:
resale_data_latlong

,year_sold,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,Latitude,Longitude
0,726468,1 ROOM,10 TO 12,31.0,improved,719163,9000.0,79,1.360174,103.852629
1,726468,1 ROOM,04 TO 06,31.0,improved,719163,6000.0,79,1.360174,103.852629
2,726468,1 ROOM,10 TO 12,31.0,improved,719163,8000.0,79,1.360174,103.852629
3,726468,1 ROOM,07 TO 09,31.0,improved,719163,6000.0,79,1.360174,103.852629
4,726468,3 ROOM,04 TO 06,73.0,new generation,719163,47200.0,79,1.360174,103.852629
...,...,...,...,...,...,...,...,...,...,...
923892,738977,EXECUTIVE,07 TO 09,181.0,apartment,719163,1080000.0,67,1.414234,103.838955
923893,738977,EXECUTIVE,07 TO 09,142.0,apartment,719163,780888.0,63,1.429532,103.849485
923894,738977,EXECUTIVE,04 TO 06,146.0,apartment,719163,830000.0,63,1.426451,103.837204
923895,738977,EXECUTIVE,04 TO 06,146.0,maisonette,719163,880000.0,63,1.418230,103.833577


In [133]:
# performance_of_models = []

# ## DECISION TREE REGRESSOR
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error

# dtr = DecisionTreeRegressor(min_samples_leaf=0.10)
# test_data.loc[:,"flat_type"] = ord_encoder_flattype.transform(test_data[["flat_type"]])
# test_data.loc[:,"storey_range"] = ord_encoder_storey.transform(test_data[["storey_range"]])
# flat_model_1hot_test = cat_encoder.transform(test_data[["flat_model"]])
# test_data = test_data.drop("flat_model",axis=1)
# test_target = test_data["resale_price"].values
# test_data = test_data.drop(["resale_price"],axis=1)
# test_features = np.concatenate((test_data.values,flat_model_1hot_test.toarray()),axis=1)
# dtr.fit(features, target)


# dt_test_predict = dtr.predict(test_features)

# dt_test_mse = mean_squared_error(dt_test_predict, test_target)
# dt_test_rmse = dt_test_mse**(1/2)

# dt_train_predict = dtr.predict(features)
# dt_train_mse = mean_squared_error(dt_train_predict, target)
# dt_train_rmse = dt_train_mse**(1/2)

# dt_row = {'Model':'Decision Tree','train_rmse':dt_train_rmse,'test_rmse':dt_test_rmse}
# performance_of_models.append(dt_row)

# #RANDOM FOREST REGRESSOR 1hot
# from sklearn.ensemble import RandomForestRegressor
# rfr = RandomForestRegressor(n_estimators=500, max_leaf_nodes = 10, random_state=42)
# rfr.fit(features, target)

# rfr_test_predict = rfr.predict(test_features)
# rfr_test_mse = mean_squared_error(rfr_test_predict, test_target)
# rfr_test_rmse = rfr_test_mse**(1/2)

# rfr_train_predict = rfr.predict(features)
# rfr_train_mse = mean_squared_error(rfr_train_predict, target)
# rfr_train_rmse = rfr_train_mse**(1/2)

# rfr_row = {'Model':'Random Forest 1hot','train_rmse':rfr_train_rmse,'test_rmse':rfr_test_rmse}
# performance_of_models.append(rfr_row)

# #RANDOM FOREST REGRESSOR freq
# copy_resale_data_latlong = resale_data_latlong.copy()
# copy_resale_data_dict = copy_resale_data_latlong.flat_model.value_counts().to_dict()
# copy_resale_data_latlong.flat_model = copy_resale_data_latlong.flat_model.map(copy_resale_data_dict)


# from sklearn.model_selection import StratifiedShuffleSplit
# splitter = StratifiedShuffleSplit(n_splits=1,test_size = 0.2,random_state=42)
# strat_splits = []
# for train_index, test_index in splitter.split(copy_resale_data_latlong, copy_resale_data_latlong['year_sold']):
#   copy_train_data = copy_resale_data_latlong.iloc[train_index]
#   copy_test_data = copy_resale_data_latlong.iloc[test_index]
#   strat_splits.append([copy_train_data, copy_test_data])


# from sklearn.preprocessing import OrdinalEncoder
# flat_type_order = [['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM','EXECUTIVE','MULTI-GENERATION']]
# storey_range_order = [['01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09', '10 TO 12', '11 TO 15','13 TO 15', '16 TO 18', '16 TO 20', '19 TO 21',
#                        '21 TO 25', '22 TO 24', '25 TO 27','26 TO 30', '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40', '37 TO 39','40 TO 42',
#                        '43 TO 45', '46 TO 48', '49 TO 51']]
# copy_ord_encoder_flattype = OrdinalEncoder(categories=flat_type_order)
# copy_ord_encoder_storey = OrdinalEncoder(categories=storey_range_order)
# copy_train_data.loc[:,"flat_type"] = copy_ord_encoder_flattype.fit_transform(copy_train_data[["flat_type"]])
# copy_train_data.loc[:,"storey_range"] = copy_ord_encoder_storey.fit_transform(copy_train_data[["storey_range"]])


# from sklearn.ensemble import RandomForestRegressor

# target_freq = copy_train_data['resale_price'].values
# features_freq = copy_train_data.drop(["resale_price"],axis=1).values

# rfr_freq = RandomForestRegressor(n_estimators=500, max_leaf_nodes = 10, random_state=42)
# rfr_freq.fit(features_freq, target_freq)

# copy_test_data.loc[:,"flat_type"] = copy_ord_encoder_flattype.transform(copy_test_data[["flat_type"]])
# copy_test_data.loc[:,"storey_range"] = copy_ord_encoder_storey.transform(copy_test_data[["storey_range"]])
# copy_test_target = copy_test_data["resale_price"].values
# copy_test_features = copy_test_data.drop(["resale_price"],axis=1).values

# rfr_freq_test_predict = rfr_freq.predict(copy_test_features)
# rfr_freq_test_mse = mean_squared_error(rfr_freq_test_predict, copy_test_target)
# rfr_freq_test_rmse = rfr_freq_test_mse**(1/2)

# rfr_freq_train_predict = rfr_freq.predict(features_freq)
# rfr_freq_train_mse = mean_squared_error(rfr_freq_train_predict, target_freq)
# rfr_freq_train_rmse = rfr_freq_train_mse**(1/2)

# rfr_freq_row = {'Model':'Random Forest frequency','train_rmse':rfr_freq_train_rmse,'test_rmse':rfr_freq_test_rmse}
# performance_of_models.append(rfr_freq_row)


# ##Linear Regression
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# scaler = StandardScaler()
# scaled_train_data = scaler.fit_transform(train_data)
# scaled_features = np.concatenate((scaled_train_data,flat_model_1hot.toarray()),axis=1)
# lr = LinearRegression()
# lr.fit(scaled_features, target)

# scaled_test_data = scaler.transform(test_data)
# scaled_test_features = np.concatenate((scaled_test_data,flat_model_1hot_test.toarray()),axis=1)
# lr_test_predict = lr.predict(scaled_test_features)
# lr_test_mse = mean_squared_error(lr_test_predict, test_target)
# lr_test_rmse = lr_test_mse**(1/2)

# lr_train_predict = lr.predict(scaled_features)
# lr_train_mse = mean_squared_error(lr_train_predict, target)
# lr_train_rmse = lr_train_mse**(1/2)

# lr_row = {'Model':'Linear Regression','train_rmse':lr_train_rmse,'test_rmse':lr_test_rmse}
# performance_of_models.append(lr_row)

# from sklearn.ensemble import BaggingRegressor
# br = BaggingRegressor(estimator=lr, n_estimators=300, oob_score=True)
# br.fit(scaled_features, target)
# bagging_test_predict = br.predict(scaled_test_features)
# bagging_test_mse = mean_squared_error(bagging_test_predict, test_target)
# bagging_test_rmse = bagging_test_mse**(1/2)

# bagging_train_predict = br.predict(scaled_features)
# bagging_train_mse = mean_squared_error(bagging_train_predict, target)
# bagging_train_rmse = bagging_train_mse**(1/2)

# bagging_row = {'Model':'Bagging Regressor','train_rmse':bagging_train_rmse,'test_rmse':bagging_test_rmse}
# performance_of_models.append(bagging_row)

# performance_of_models = pd.DataFrame(performance_of_models)
# performance_of_models

,Model,train_rmse,test_rmse
0,Decision Tree,93855.246297,94104.068272
1,Random Forest 1hot,83466.564363,83684.385855
2,Random Forest frequency,83466.564363,83684.385855
3,Linear Regression,74828.438295,74847.065215
4,Bagging Regressor,74828.445090,74847.081767


## XGBoost

XGBoost is one of the most powerful algorithms. Let's check if this can reduce RMSE too. We will train 2 XGBoost models. One with l1(lasso) and l2(ridge) regression.
Additionally, we will be using cross validation and will try to get the best value for the hyperparameters(alpha and lambda).

In [134]:
X,y = resale_data_latlong.loc[:, ~resale_data_latlong.columns.isin(['resale_price'])], resale_data_latlong.loc[:,"resale_price"]
from sklearn.preprocessing import OrdinalEncoder
flat_type_order = [['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM','EXECUTIVE','MULTI-GENERATION']]
storey_range_order = [['01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09', '10 TO 12', '11 TO 15','13 TO 15', '16 TO 18', '16 TO 20', '19 TO 21',
                       '21 TO 25', '22 TO 24', '25 TO 27','26 TO 30', '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40', '37 TO 39','40 TO 42',
                       '43 TO 45', '46 TO 48', '49 TO 51']]
ord_encoder_flattype = OrdinalEncoder(categories=flat_type_order)
ord_encoder_storey = OrdinalEncoder(categories=storey_range_order)
X.loc[:,"flat_type"] = ord_encoder_flattype.fit_transform(X[["flat_type"]])
X.loc[:,"storey_range"] = ord_encoder_storey.fit_transform(X[["storey_range"]])

## ONE HOT ENCODING OF
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder(handle_unknown="ignore")
flat_model_1hot = cat_encoder.fit_transform(X[["flat_model"]])
X = X.drop("flat_model",axis =1)
X = np.concatenate((X.values,flat_model_1hot.toarray()),axis = 1)

import xgboost as xgb
xg_matrix = xgb.DMatrix(data=X, label=y.values)
reg_params = [0.0001, 0.001, 0.01, 0.1, 1]
params = {'objective':'reg:squarederror','max_depth':5}
rmses_l1 = []
for reg in reg_params:

    params["alpha"] = reg

    cv_results_rmse = xgb.cv(dtrain=xg_matrix, params=params, nfold=3, num_boost_round=10, metrics="rmse", as_pandas=True, seed=123)

    rmses_l1.append(cv_results_rmse["test-rmse-mean"].tail(1).values[0])

print("Best rmse as a function of l1:")
print(pd.DataFrame(list(zip(reg_params, rmses_l1)), columns=["l1", "rmse"]))

import xgboost as xgb
xg_matrix_2 = xgb.DMatrix(data=X, label=y.values)
reg_params = [0.01, 0.1, 1, 10]
params = {'objective':'reg:squarederror','max_depth':5}
rmses_l2 = []
for reg in reg_params:

    params["lambda"] = reg

    cv_results_rmse = xgb.cv(dtrain=xg_matrix, params=params, nfold=3, num_boost_round=10, metrics="rmse", as_pandas=True, seed=123)

    rmses_l2.append(cv_results_rmse["test-rmse-mean"].tail(1).values[0])

print("Best rmse as a function of l2:")
print(pd.DataFrame(list(zip(reg_params, rmses_l2)), columns=["l2", "rmse"]))


Best rmse as a function of l1:
       l1          rmse
0  0.0001  45950.855092
1  0.0010  45950.855092
2  0.0100  45950.855093
3  0.1000  45950.855110
4  1.0000  45950.855164
Best rmse as a function of l2:
      l2          rmse
0   0.01  46007.266136
1   0.10  46008.943039
2   1.00  45950.855092
3  10.00  46067.028122


###l1(lasso regression) has much better RMSE score than other models at alpha=0.001.
We will be selecting this to train our final model.

In [135]:
params = {'objective': 'reg:squarederror', 'max_depth': 5, "alpha":0.001}
final_model = xgb.train(params=params, dtrain=xg_matrix, num_boost_round=10)

### Let's create the streamlit app.
The user must be able to select the town, storey range, street, flat type, flat model. For this, we will create drop down options. Based on selection of the town, the streets drop down will be updated.
Additionally, they must be able to enter the floor area.
All these entries will be transformed with the help of our encoders and used to predict the price which will be displayed.

To run this app in colab, please follow these [steps](https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399/4).


In [136]:
import streamlit as st

def get_street_name(selected_town):
    if selected_town == 'ANG MO KIO':
        return ['ANG MO KIO ST 31','ANG MO KIO ST 32','ANG MO KIO AVE 10','ANG MO KIO AVE 2','ANG MO KIO AVE 8','ANG MO KIO ST 44','ANG MO KIO ST 51','ANG MO KIO AVE 1','ANG MO KIO ST 21','ANG MO KIO AVE 6','ANG MO KIO AVE 9','ANG MO KIO AVE 4','ANG MO KIO AVE 3','ANG MO KIO ST 61', 'ANG MO KIO AVE 5','ANG MO KIO ST 52','ANG MO KIO ST 11']
    elif selected_town == "BEDOK":
        return ['BEDOK RESERVOIR RD','BEDOK RESERVOIR CRES','EAST COAST RD','CHAI CHEE RD','BEDOK NTH ST 4', 'BEDOK RESERVOIR VIEW','BEDOK NTH AVE 4','JLN TENAGA', 'BEDOK STH AVE 1',
'CHAI CHEE AVE','CHAI CHEE DR','BEDOK STH AVE 3','BEDOK NTH AVE 2','BEDOK STH RD','LENGKONG TIGA','BEDOK NTH AVE 3','NEW UPP CHANGI RD','BEDOK CTRL','CHAI CHEE ST','BEDOK NTH AVE 1','BEDOK NTH ST 1',
  'BEDOK STH AVE 2','JLN DAMAI','BEDOK NTH RD','BEDOK NTH ST 3','BEDOK NTH ST 2']
    elif selected_town == "BISHAN":
        return ['BISHAN ST 11','BISHAN ST 23','SIN MING RD','BRIGHT HILL DR','SIN MING AVE','BISHAN ST 22','BISHAN ST 13','BISHAN ST 24','BISHAN ST 12','SHUNFU RD']
    elif selected_town == "BUKIT BATOK":
        return ['BT BATOK WEST AVE 9','BT BATOK ST 51','BT BATOK CTRL','BT BATOK ST 32','BT BATOK WEST AVE 4','BT BATOK ST 25','BT BATOK EAST AVE 5','BT BATOK ST 24','BT BATOK ST 22','BT BATOK ST 52','BT BATOK EAST AVE 6','BT BATOK ST 33','BT BATOK ST 11','BT BATOK WEST AVE 5','BT BATOK ST 31','BT BATOK WEST AVE 6','BT BATOK ST 34','BT BATOK EAST AVE 4','BT BATOK EAST AVE 3','HILLVIEW AVE','BT BATOK WEST AVE 7','BT BATOK WEST AVE 8','BT BATOK WEST AVE 2','BT BATOK ST 21']
    elif selected_town == "BUKIT MERAH":
        return ['DEPOT RD', 'SPOTTISWOODE PK RD', 'KIM TIAN RD', 'JLN MEMBINA BARAT', 'TELOK BLANGAH ST 31', 'HENDERSON CRES', 'MOH GUAN TER', 'REDHILL RD', 'JLN BT MERAH', 'HAVELOCK RD', 'KIM TIAN PL', 'TELOK BLANGAH WAY', 'TIONG BAHRU RD', 'SILAT AVE', 'SENG POH RD', 'LOWER DELTA RD', 'KIM PONG RD', 'TELOK BLANGAH HTS', 'REDHILL LANE', 'TELOK BLANGAH CRES', 'CANTONMENT CL', 'BT MERAH LANE 1', 'KIM CHENG ST', 'NILE RD', 'BT MERAH CTRL', 'EVERTON PK', 'REDHILL CL', 'JLN MEMBINA', 'TELOK BLANGAH DR', 'LENGKOK BAHRU', 'TAMAN HO SWEE', 'BT PURMEI RD', 'JLN BT HO SWEE', 'JLN KLINIK', 'JLN RUMAH TINGGI', 'BT MERAH VIEW', 'ALEXANDRA RD', 'HOY FATT RD', 'BEO CRES', 'LIM LIAK ST', 'HENDERSON RD', 'TELOK BLANGAH RISE', 'KG BAHRU HILL', 'ZION RD', 'BOON TIONG RD', 'INDUS RD', 'DELTA AVE']
    elif selected_town == "BUKIT TIMAH":
        return ['EMPRESS RD', 'FARRER RD', 'TOH YI DR', "QUEEN'S RD"]
    elif selected_town == "CENTRAL AREA":
        return ['SMITH ST', 'OUTRAM HILL', 'UPP CROSS ST', 'SHORT ST', 'KLANG LANE', 'CHIN SWEE RD', 'CHANDER RD', 'TG PAGAR PLAZA', 'ROWELL RD', 'VEERASAMY RD', 'OUTRAM PK', 'NEW MKT RD', 'BUFFALO RD', 'KELANTAN RD', 'QUEEN ST', 'ROCHOR RD', 'JLN BERSEH', 'SELEGIE RD', 'SAGO LANE', 'CANTONMENT RD', 'KRETA AYER RD', 'JLN KUKOH', 'WATERLOO ST', 'BAIN ST']
    elif selected_town == "CHOA CHU KANG":
        return ['CHOA CHU KANG AVE 3', 'TECK WHYE AVE', 'CHOA CHU KANG AVE 4', 'CHOA CHU KANG AVE 5', 'CHOA CHU KANG CRES', 'CHOA CHU KANG AVE 2', 'CHOA CHU KANG ST 53', 'TECK WHYE LANE', 'JLN TECK WHYE', 'CHOA CHU KANG NTH 5', 'CHOA CHU KANG NTH 7', 'CHOA CHU KANG NTH 6', 'CHOA CHU KANG LOOP', 'CHOA CHU KANG ST 54', 'CHOA CHU KANG CTRL', 'CHOA CHU KANG ST 52', 'KEAT HONG LINK', 'CHOA CHU KANG ST 64', 'TECK WHYE CRES', 'CHOA CHU KANG ST 51', 'CHOA CHU KANG AVE 1', 'CHOA CHU KANG AVE 7', 'CHOA CHU KANG DR', 'CHOA CHU KANG ST 62', 'KEAT HONG CL']
    elif selected_town == "CLEMENTI":
        return ['CLEMENTI WEST ST 1', 'CLEMENTI ST 13', 'WEST COAST RD', 'CLEMENTI WEST ST 2', 'CLEMENTI AVE 2', 'CLEMENTI ST 14', 'CLEMENTI ST 11', 'CLEMENTI AVE 3', 'CLEMENTI AVE 6', 'CLEMENTI ST 12', 'CLEMENTI AVE 1', 'CLEMENTI AVE 5', 'WEST COAST DR', 'CLEMENTI AVE 4', "C'WEALTH AVE WEST"]
    elif selected_town == "GEYLANG":
        return ['GEYLANG EAST AVE 1', 'HAIG RD', 'UBI AVE 1', 'CIRCUIT RD', 'ALJUNIED AVE 2', 'JLN PASAR BARU', 'GEYLANG SERAI', 'EUNOS CRES', 'SIMS DR', 'JOO CHIAT RD', 'GEYLANG EAST AVE 2', 'ALJUNIED CRES', 'OLD AIRPORT RD', 'GEYLANG EAST CTRL', 'BALAM RD', 'JLN TIGA', 'DAKOTA CRES', 'JLN DUA', 'CASSIA CRES', 'SIMS AVE', 'EUNOS RD 5', 'SIMS PL', 'MACPHERSON LANE', 'ALJUNIED RD', 'PIPIT RD', 'PAYA LEBAR WAY', 'PINE CL']
    elif selected_town == "HOUGANG":
        return ['HOUGANG ST 11', 'HOUGANG ST 61', 'HOUGANG AVE 4', 'BUANGKOK LINK', 'HOUGANG AVE 3', 'BUANGKOK GREEN', 'HOUGANG ST 91', 'UPP SERANGOON RD', 'HOUGANG ST 22', 'HOUGANG ST 21', 'HOUGANG AVE 1', 'HOUGANG ST 92', 'UPP SERANGOON VIEW', 'HOUGANG AVE 9', 'HOUGANG ST 32', 'UPP SERANGOON CRES', 'HOUGANG CTRL', 'HOUGANG AVE 5', 'LOR AH SOO', 'HOUGANG ST 31', 'HOUGANG AVE 10', 'HOUGANG AVE 2', 'HOUGANG ST 52', 'HOUGANG AVE 6', 'HOUGANG AVE 7', 'BUANGKOK CRES', 'HOUGANG ST 51', 'HOUGANG AVE 8']
    elif selected_town == "JURONG EAST":
        return ['JURONG EAST ST 24', 'JURONG EAST ST 21', 'JURONG EAST ST 32', 'TOH GUAN RD', 'JURONG EAST AVE 1', 'PANDAN GDNS', 'TEBAN GDNS RD', 'JURONG EAST ST 13', 'JURONG EAST ST 31']
    elif selected_town == "JURONG WEST":
        return ['JURONG WEST ST 65', 'HU CHING RD', 'KANG CHING RD', 'YUNG SHENG RD', 'BOON LAY DR', 'YUNG LOH RD', 'YUNG KUANG RD', 'BOON LAY PL', 'JURONG WEST AVE 3', 'TAO CHING RD', 'JURONG WEST ST 52', 'JURONG WEST AVE 1', 'JURONG WEST CTRL 3', 'TAH CHING RD', 'JURONG WEST ST 81', 'JURONG WEST AVE 5', 'JURONG WEST CTRL 1', 'JURONG WEST ST 91', 'YUNG HO RD', 'YUNG PING RD', 'YUAN CHING RD', 'JURONG WEST ST 24', 'YUNG AN RD', 'JURONG WEST ST 62', 'CORPORATION DR', 'JURONG WEST ST 75', 'JURONG WEST ST 64', 'JURONG WEST ST 41', 'JURONG WEST ST 51', 'JURONG WEST ST 42', 'JURONG WEST ST 61', 'JURONG WEST ST 73', 'JURONG WEST ST 25', 'JURONG WEST ST 74', 'BOON LAY AVE', 'JURONG WEST ST 72', 'JURONG WEST ST 93', 'JURONG WEST ST 71', 'JURONG WEST ST 92', 'HO CHING RD']
    elif selected_town == "KALLANG/WHAMPOA":
        return ['RACE COURSE RD', 'KALLANG BAHRU', 'BENDEMEER RD', 'KG KAYU RD', 'AH HOOD RD', "JLN MA'MOR", 'DORSET RD', 'BEACH RD', 'LOR LIMAU', 'WHAMPOA STH', 'JLN DUSUN', 'KG ARANG RD', 'WHAMPOA DR', 'UPP BOON KENG RD', 'JELLICOE RD', 'GLOUCESTER RD', 'CAMBRIDGE RD', 'OWEN RD', 'TOWNER RD', 'TESSENSOHN RD', 'FRENCH RD', 'FARRER PK RD', "ST. GEORGE'S RD", 'MOULMEIN RD', 'MCNAIR RD', "ST. GEORGE'S LANE", 'CRAWFORD LANE', 'JLN TENTERAM', "KING GEORGE'S AVE", 'WHAMPOA RD', 'GEYLANG BAHRU', 'BOON KENG RD', 'LOR 3 GEYLANG', 'JLN BAHAGIA', 'NTH BRIDGE RD', 'JLN BATU', 'JLN RAJAH', 'WHAMPOA WEST', 'KENT RD']
    elif selected_town == "MARINE PARADE":
        return ['MARINE PARADE CTRL', 'MARINE DR', 'MARINE CRES', 'MARINE TER']
    elif selected_town == "QUEENSTOWN":
        return ['HOLLAND DR', "C'WEALTH AVE", "QUEEN'S CL", 'MEI LING ST', 'GHIM MOH LINK', 'HOLLAND CL', 'QUEENSWAY', "C'WEALTH DR", 'STRATHMORE AVE', 'DOVER RD', 'MARGARET DR', 'DOVER CL EAST', 'TANGLIN HALT RD', 'GHIM MOH RD', 'STIRLING RD', 'DAWSON RD', "C'WEALTH CRES", 'DOVER CRES', "C'WEALTH CL", 'CLARENCE LANE', 'HOLLAND AVE']
    elif selected_town == "SENGKANG":
        return ['BUANGKOK STH FARMWAY 1', 'SENGKANG EAST RD', 'COMPASSVALE LINK', 'ANCHORVALE LINK', 'COMPASSVALE LANE', 'COMPASSVALE WALK', 'ANCHORVALE ST', 'RIVERVALE WALK', 'FERNVALE RD', 'RIVERVALE DR', 'SENGKANG EAST AVE', 'COMPASSVALE BOW', 'FERNVALE LANE', 'SENGKANG WEST AVE', 'COMPASSVALE CRES']

def get_lat(user_street, user_town):
  return coordinates_df.loc[coordinates_df['full_address'].str.contains(user_street+','+user_town, regex=True),'Latitude'].iloc[0]

def get_long(user_street, user_town):
  return coordinates_df.loc[coordinates_df['full_address'].str.contains(user_street+','+user_town, regex=True),'Longitude'].iloc[0]

def get_storeyrange(user_storey):
  storey_df = pd.DataFrame({'storey_range':[user_storey]})
  return ord_encoder_storey.transform(storey_df[['storey_range']])[0][0]

def get_flattype(user_storey):
  flattype_df = pd.DataFrame({'flat_type':[user_storey]})
  return ord_encoder_flattype.transform(flattype_df[['flat_type']])[0][0]

def get_flatmodel(user_model):
  flatmodel_df = pd.DataFrame({'flat_model':[user_model]})
  return cat_encoder.transform(flatmodel_df[['flat_model']])

def main():
  st.title("Singapore Resale Flat Price Estimator")

    # Dropdown options
  town_options = ('ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH','BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG', 'CLEMENTI',
       'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST','KALLANG/WHAMPOA', 'MARINE PARADE', 'QUEENSTOWN', 'SENGKANG','SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN','LIM CHU KANG', 'SEMBAWANG', 'BUKIT PANJANG', 'PASIR RIS','PUNGGOL')
  flat_type_options = ('1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE', 'MULTI-GENERATION')
  flat_model_options = ('improved', 'new generation', 'model a', 'standard', 'simplified','model a-maisonette', 'apartment', 'maisonette', 'terrace',
       '2-room', 'improved-maisonette', 'multi generation', 'premium apartment', 'adjoined flat', 'premium maisonette','model a2', 'dbss', 'type s1',
        'type s2', 'premium apartment loft','3gen')
  storey_range_options = ('01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09', '10 TO 12', '11 TO 15','13 TO 15', '16 TO 18', '16 TO 20', '19 TO 21',
                       '21 TO 25', '22 TO 24', '25 TO 27','26 TO 30', '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40', '37 TO 39','40 TO 42',
                       '43 TO 45', '46 TO 48', '49 TO 51')

    # Dropdowns
  selected_town = st.selectbox("Select Town", town_options)
  selected_street = st.selectbox("Select Town", get_street_name(selected_town))
  selected_flat_type = st.selectbox("Select Flat Type", flat_type_options)
  selected_flat_model = st.selectbox("Select Flat Model", flat_model_options)
  user_lease_commence_date = st.number_input("Lease commence date(yyyy-mm-dd):-", value = 0)
  selected_storey_range = st.selectbox("Select Storey Range", storey_range_options)
  floor_area = st.number_input("Floor Area (sqm)", value = 0)
  user_data = {
    "town": [selected_town],
    "street": [selected_street],
    "flat_type": [selected_flat_type],
    "flat_model": [selected_flat_model],
    "storey_range": [selected_storey_range],
    "floor_area_sqm": [floor_area],
    "lease_commence_date":[user_lease_commence_date]}
  user_df = pd.DataFrame(user_data)
  if st.button("Get estimate:"):
   user_df['Latitude'] = get_lat(selected_street, selected_town)
   user_df['Longitude'] = get_long(selected_street, selected_town)
   user_df['storey_range'] = get_storeyrange(selected_storey_range)
   user_df['flat_type'] = get_flattype(selected_flat_type)
   flat_model_arr = get_flatmodel(selected_flat_model)
   user_df['year_sold'] = pd.to_datetime('2024')
   year  = user_df['year_sold'].dt.year
   user_df['lease_commence_date'] = pd.to_datetime(user_df['lease_commence_date'], format='%Y-%m-%d')
   lease_year = user_df['lease_commence_date'].dt.year
   user_df['remaining_lease'] = year - lease_year
   user_df['lease_commence_date'] = user_df['lease_commence_date'].apply(lambda x: x.toordinal())
   user_df['year_sold'] = user_df['year_sold'].apply(lambda x: x.toordinal())
   desired_col_order = ['year_sold','flat_type',	'storey_range',	'floor_area_sqm', 'lease_commence_date', 'remaining_lease', 	'Latitude',	'Longitude']
   user_df = user_df.drop(['town','street','flat_model'],axis=1)
   user_df = user_df[desired_col_order]
   user_features = np.concatenate((user_df.values,flat_model_arr.toarray()),axis=1)
   user_matrix = xgb.DMatrix(user_features)
   estimated_price = final_model.predict(user_matrix)
   st.success(f"The estimated price is: ${estimated_price[0]}")

if __name__ == '__main__':
  main()